In [130]:
import pandas as pd
import numpy as np
pd.set_option('max_rows', 200)

In [14]:
basic = pd.read_csv('./basic.csv', sep = ';')

In [16]:
basic.columns

Index(['date', 'time', 'price', 'shares', 'tax', 'fee', 'type', 'broker',
       'isin'],
      dtype='object')

In [8]:
imp = pd.read_csv('./transactions-from-23062018-to-18082022.csv',sep = ';')

In [25]:
imp

,Datum,Auftrag #,Transaktionen,Symbol,Name,ISIN,Anzahl,Stückpreis,Kosten,Aufgelaufene Zinsen,Nettobetrag,Saldo,Währung
0,11-08-2022 13:40:12,0,Dividende,AAPL,APPLE ORD,US0378331005,1.000,2.760000,0.82,0.0,1.94,42.96,USD
1,01-08-2022 14:02:35,0,Dividende,T,AT&T ORD,US00206R1023,1.000,14.990000,4.50,0.0,10.49,41.02,USD
2,12-07-2022 08:57:13,112012823,Kauf,BAS,BASF N ORD,DE000BASF111,20.000,40.900000,29.57,0.0,-847.57,1272.31,EUR
3,12-07-2022 07:41:15,0,Rückzahlung,WBD,WARNER BROS. DISCOVERY SRS A O,US9344231041,-0.064,13.055377,0.00,0.0,-,NaN,USD
4,12-07-2022 07:41:15,0,Rückzahlung,WBD,WARNER BROS. DISCOVERY SRS A O,US9344231041,1.000,0.840000,0.00,0.0,0.84,30.53,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,17-04-2020 22:16:44,0,Fx-Gutschrift Comp.,NaN,NaN,NaN,1.000,7277.380000,0.00,0.0,7277.38,0.00,USD
132,17-04-2020 22:16:44,0,Fx-Belastung Comp.,NaN,NaN,NaN,1.000,7108.570000,0.00,0.0,-7108.57,2891.43,CHF
133,17-04-2020 16:15:40,70470255,Kauf,SWDA,iSh Core MSCI Wrld USD Acc,IE00B4L5Y983,135.000,53.800000,14.38,0.0,-7277.38,-7277.38,USD
134,02-04-2020 10:50:51,69601673,Vergütung,NaN,NaN,NaN,1.000,9000.000000,0.00,0.0,9000.00,10000.00,CHF


In [143]:
mapping_col = {'Datum':'datetime',
               'Auftrag #':'Auftrag #',
               'Transaktionen':'type',
               'Symbol':'Symbol',
               'Name':'Name',
               'ISIN':'isin',
               'Anzahl':'shares',
               'Stückpreis':'price', 
               'Kosten':'fee', 
               'Aufgelaufene Zinsen':'Aufgelaufene Zinsen', 
               'Nettobetrag':'Nettobetrag',
               'Saldo':'Saldo',
               'Währung':'originalcurrency'}

mapping_transactions = {'Dividende':'Dividend',
                        'Kauf':'Buy', 
                        'Rückzahlung':'Rückzahlung', 
                        'Depotgebühren':'Depotgebühren',
                        'Fx-Gutschrift Comp.':'TransferIn', 
                        'Fx-Belastung Comp.':'TransferOut', 
                        'Spin off':'Spin off',
                        'Capital Gain':'Capital Gain', 
                        'Verkauf':'Sell', 
                        'Forex-Gutschrift':'Forex-Gutschrift', 
                        'Forex-Belastung':'Forex-Belastung',
                        'Zins':'Zins',
                        'Interne Titelumbuchung':'Interne Titelumbuchung',
                        'Vergütung':'TransferIn',
                        'Berichtigung Börsengeb.':'Berichtigung Börsengeb.'}
keep_columns = ['datetime',
               'type',
               'isin',
               'shares',
               'price',
               'fee',
               'originalcurrency']

In [175]:
exp = imp.rename(columns=mapping_col)[keep_columns]
exp.replace(to_replace = mapping_transactions.keys(), value = mapping_transactions.values(), inplace = True)
exp['holding'] = ''

exp.query("type in ['TransferIn','TransferOut'] and originalcurrency == 'CHF'").holding
exp.loc[((exp.type == 'TransferIn')|(exp.type == 'TransferOut')) & (exp.originalcurrency == 'CHF'),'holding'] = 'hld_62fe9bf476362ac5e8b88ed5'
exp.loc[((exp.type == 'TransferIn')|(exp.type == 'TransferOut')) & (exp.originalcurrency == 'USD'),'holding'] = 'hld_62fe9c0476362ac5e8b88ed8' # USD
exp.loc[((exp.type == 'TransferIn')|(exp.type == 'TransferOut')) & (exp.originalcurrency == 'EUR'),'holding'] = 'hld_62fe9c1398683c34ff685ac2' # EUR

exp['datetime'] = pd.to_datetime(exp['datetime'], format='%d-%m-%Y %H:%M:%S')
exp['datetime'] = exp["datetime"].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
exp['currency'] = exp['originalcurrency']
exp['tax'] = 0.0
port = exp[exp['type'].isin(['Buy','Sell','Dividend','TransferIn','TransferOut'])]
cash = exp[exp['type'].isin(['TransferIn','TransferOut'])]
cash = cash[['datetime','type','shares','price','fee','holding','tax']]

cash.to_csv('./cash.csv',sep=';', index = False)

exp.to_csv('./portfolio.csv',sep=';')